In [2]:
import numpy as np
import gurobipy as gp
import pandas as pd

## Picking 5 stocks and measuring their performance

#### Data Loading and Preprocessing

In [3]:
stock19 = pd.read_csv('stocks2019.csv')
stock20 = pd.read_csv('stocks2020.csv')

stock19_noIDX = stock19.iloc[:,2:].pct_change().dropna()
stock20_noIDX = stock20.iloc[:,2:].pct_change().dropna()

stock19_index = stock19.iloc[:,1].pct_change().dropna()
stock20_index = stock20.iloc[:,1].pct_change().dropna()



Insights on why we're taking pct_change

In [4]:
# number of stocks in index

n = stock19.shape[1] - 2

In [5]:
stock19_noIDX = stock19.iloc[:,2:].pct_change().dropna()
corr_mat = stock19_noIDX.corr()
p = corr_mat.values

In [6]:
m=5 # number of stocks in the portfolio

In [7]:
stockmod = gp.Model()
stockmodX = stockmod.addMVar((n,n),vtype = 'B')
stockmodY = stockmod.addMVar(n,vtype = 'B')

Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-22


In [8]:
stockmod.setObjective(gp.quicksum(p[i][j]*stockmodX[i][j] for i in range(n) for j in range(n)),gp.GRB.MAXIMIZE)

In [9]:
constr1 = stockmod.addConstr(gp.quicksum(stockmodY[i] for i in range(n)) == m)
constr2 = stockmod.addConstrs((gp.quicksum(stockmodX[i][j] for j in range(n)) == 1) for i in range(n))
constr3 = stockmod.addConstrs((stockmodX[i][j] <= stockmodY[j]) for i in range(n) for j in range(n))

In [10]:
stockmod.Params.OutputFlag = 0 # tell gurobi to shut up!!
stockmod.optimize()

In [11]:
selected_stocks = [corr_mat.columns[j] for j in range(n) if stockmodY[j].X == 1]

print(f'The stocks we chose are {selected_stocks}')

The stocks we chose are ['LBTYK', 'MXIM', 'MSFT', 'VRTX', 'XEL']


#### Calculating weights for the selected stocks

In [12]:
index_rt = stock19_index.values
t = stock19_noIDX.shape[0]

In [13]:
rt_df = stock19_noIDX.iloc[:,list(np.where(stockmodY.x)[0])]
rt = rt_df.values

In [14]:
wmod = gp.Model()
wmodX = wmod.addMVar(m,vtype='C',lb=0)
wmodY = wmod.addMVar(t,vtype='C')

In [15]:
wmod.setObjective(gp.quicksum(wmodY[i] for i in range(t)))

In [16]:
constr1 = wmod.addConstr(gp.quicksum(wmodX[i] for i in range(m)) == 1)
constr2 = wmod.addConstrs((wmodY[i] >= (index_rt[i] - gp.quicksum(wmodX[j]*rt[i][j] for j in range(m)))) for i in range(t))
constr3 = wmod.addConstrs((wmodY[i] >= -1*(index_rt[i] - gp.quicksum(wmodX[j]*rt[i][j] for j in range(m)))) for i in range(t))

In [17]:
wmod.Params.OutputFlag = 0 # tell gurobi to shut up!!
wmod.optimize()

In [18]:
print("Error of our portfolio is :",wmod.ObjVal)

Error of our portfolio is : 0.7891782824631473


In [19]:
for i in range(m):
    print(f'The weight for {rt_df.columns[i]} is {wmodX[i].X}')

The weight for LBTYK is 0.04886174835252491
The weight for MXIM is 0.21038806005665553
The weight for MSFT is 0.5803519807862964
The weight for VRTX is 0.07119021516911037
The weight for XEL is 0.08920799563541283


#### Tracking our portfolio with 2020 index returns

In [20]:
return_set = stock20_noIDX[selected_stocks]
index_return = stock20_index
T = len(return_set)
total = 0
for t in range(T):
    total += abs(index_return.iloc[t] - sum(wmodX[i].X * return_set.iloc[t,i] for i in range(m)))

print(f'Error would be {total}')

Error would be 1.1124373455076468


## Q3

## Q4

In [22]:
error_2019 = []
error_2020 = []
t_time = 3600
M = 2 #however, smallest big M could be >1 because weights cannot be more than 1
t = stock19_noIDX.shape[0]
rt = stock19_noIDX.values
results = pd.DataFrame(columns=['m', 'selected_stocks', 'weights', 'error_2019','error_2020'])

m_list = [i for i in range(10, 101, 10)]

for m in m_list:
    wmod = gp.Model()
    wmodX = wmod.addMVar(n,vtype='C',lb=0)
    wmodO = wmod.addMVar(t,vtype='C')
    wmodY = wmod.addMVar(n,vtype='B')


    wmod.setObjective(gp.quicksum(wmodO[i] for i in range(t)))

    constr1 = wmod.addConstr(gp.quicksum(wmodX[i] for i in range(n)) == 1)
    constr2 = wmod.addConstrs((wmodO[i] >= (index_rt[i] - gp.quicksum(wmodX[j]*rt[i][j] for j in range(n)))) for i in range(t))
    constr3 = wmod.addConstrs((wmodO[i] >= -1*(index_rt[i] - gp.quicksum(wmodX[j]*rt[i][j] for j in range(n)))) for i in range(t))
    constr4 = wmod.addConstrs((wmodX[i] <= M*wmodY[i]) for i in range(n))
    constr5 = wmod.addConstr(gp.quicksum(wmodY[i] for i in range(n)) == m)

    wmod.Params.OutputFlag = 0 # tell gurobi to shut up!!
    wmod.setParam('TimeLimit', t_time) # Stopping gurobi after 1 hour
    wmod.optimize()

    weights = [wmodX[i].X for i in range(n) if wmodY[i].X == 1]
    error_2019 = wmod.ObjVal

    selected_stocks = [corr_mat.columns[j] for j in range(n) if wmodY[j].X == 1]

    return_set = stock20_noIDX
    index_return = stock20_index
    T = len(return_set)
    total = 0
    for t in range(T):
        total += abs(index_return.iloc[t] - sum(wmodX[i].X * return_set.iloc[t,i] for i in range(n)))

    error_2020 = total

    new_row = {'m': m, 'selected_stocks': selected_stocks, 'weights': weights, 'error_2019': error_2019, 'error_2020': error_2020}
    results.loc[len(results)] = new_row

results.to_csv('result_5.csv', index=False)
